In [2]:
import pandas as pd
import urllib
from sklearn import metrics

In [35]:
# url = "https://minio.cvmart.net/user-file/23708/601a4f6ce5784700850b859d366b7a26.csv"
# urllib.request.urlretrieve(url, "train_data_noWordCut.csv")

In [3]:
pd_all = pd.read_csv("train_data_noWordCut.csv")

In [4]:
pd_all = pd.DataFrame(pd_all)

In [5]:
pd_all.head(10)

,text,label
0,{%#静下心来听音乐#%} “书中自有黄金屋，书中自有颜如玉”。沿着岁月的长河跋涉，或是风光...,1
1,这是英超被黑的最惨的一次[二哈][二哈]十几年来，中国只有孙继海，董方卓，郑智，李铁登陆过英...,0
2,【俞曾港：专业聚焦和产业链延伸是企业“走出去”根本要义】中国远洋海运集团副总经理俞曾港4月2...,1
3,看《流星花园》其实也还好啦，现在的观念以及时尚眼光都不一样了，或许十几年之后的人看我们的现在...,1
4,汉武帝的罪己诏的真实性尽管存在着争议，然而“轮台罪己诏”作为中国历史上第一份皇帝自我批评的文...,1
5,我努力的等着我想要的结局。但在某个时刻，我感觉到似乎早已有了结局。不知是我没有感觉到，还是感...,0
6,儿时的光阴永远是最美好的，从你还在襁褓中开始，一路陪伴我做琴童的日子，之后你也不可豁免滴成了...,1
7,{%#岳岳pinkray#%} 🌙{%#ONER#%} 180421 pek再看我一眼吧再...,1
8,流浪狗小黑每天坚持翻墙“非法入侵”民宅，只为了见好朋友Sooni一面，狗狗的执着打动了屋主，...,1
9,我单方面宣布：黑色和粉色是全世界最浪漫的配色 ​,1


In [7]:
# url = "https://minio.cvmart.net/user-file/23708/65afee6aa9a04bb180c2cd79ef773e44.csv"
# urllib.request.urlretrieve(url, "predict_data_noWordCut.csv")

In [6]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

device = "cpu" # not using CUDA as we do not have CUDA (AMD GPU)

In [7]:
# hyperparameters
learning_rate = 1e-3
input_size = 768
num_epoches = 10
batch_size = 100
decay_rate = 0.9

In [13]:
# Download the uploaded model

In [14]:
# url = "https://minio.cvmart.net/user-file/23708/473d59a6c58d4647b9c7a9ce945f7f95.txt"
# urllib.request.urlretrieve(url, "vocab.txt")

In [15]:
# url = "https://minio.cvmart.net/user-file/23708/ad041a10c24c47a6818f4c2379704d69.bin"
# urllib.request.urlretrieve(url, "pytorch_model.bin")

In [16]:
# url = "https://minio.cvmart.net/user-file/23708/d9e7354a821a49a2a1e0e3762c2c71df.json"
# urllib.request.urlretrieve(url, "config.json")

In [8]:
import os
from transformers import BertTokenizer, BertModel

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
MODEL_PATH = "./chinese_wwm_pytorch"

In [9]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
bert = BertModel.from_pretrained(MODEL_PATH)

Some weights of the model checkpoint at ./chinese_wwm_pytorch were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# dataset
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = pd_all["text"].tolist()
        self.label = pd_all["label"].tolist()

    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

# training set
train_data = MyDataset(pd_all)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# testing set
test_data = MyDataset(pd_all)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [11]:
# Network structure
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        out = self.sigmoid(out)
        return out

net = Net(input_size).to(device)

In [12]:
# Test set effect test
def test():
    y_pred, y_true = [], []

    with torch.no_grad():
        for words, labels in test_loader:
            tokens = tokenizer(words, padding=True)
            input_ids = torch.tensor(tokens["input_ids"]).to(device)
            attention_mask = torch.tensor(tokens["attention_mask"]).to(device)
            last_hidden_states = bert(input_ids, attention_mask=attention_mask)
            bert_output = last_hidden_states[0][:, 0]
            outputs = net(bert_output)          # Forward propagation
            outputs = outputs.view(-1)          # Flattening the output
            y_pred.append(outputs)
            y_true.append(labels)

    y_prob = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    y_pred = y_prob.clone()
    y_pred[y_pred > 0.5] = 1
    y_pred[y_pred <= 0.5] = 0
    
    print(metrics.classification_report(y_true, y_pred))
    print("准确率:", metrics.accuracy_score(y_true, y_pred))
    print("AUC:", metrics.roc_auc_score(y_true, y_prob) )

In [13]:
# Defining loss functions and optimisers
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)

In [23]:
# Iterative training
# for epoch in range(num_epoches):
#     total_loss = 0
#     for i, (words, labels) in enumerate(train_loader):
#         tokens = tokenizer(words, padding=True)
#         input_ids = torch.tensor(tokens["input_ids"]).to(device)
#         attention_mask = torch.tensor(tokens["attention_mask"]).to(device)
#         labels = labels.float().to(device)
#         with torch.no_grad():
#             last_hidden_states = bert(input_ids, attention_mask=attention_mask)
#             bert_output = last_hidden_states[0][:, 0]
#         optimizer.zero_grad()               # 梯度清零
#         outputs = net(bert_output)          # 前向传播
#         logits = outputs.view(-1)           # 将输出展平
#         loss = criterion(logits, labels)    # loss计算
#         total_loss += loss
#         loss.backward()                     # 反向传播，计算梯度
#         optimizer.step()                    # 梯度更新
#         if (i+1) % 10 == 0:
#             print("epoch:{}, step:{}, loss:{}".format(epoch+1, i+1, total_loss/10))
#             total_loss = 0
    
#     # learning_rate decay
#     scheduler.step()
    
#     # test
#     test()
    
#     # save model
#     model_path = "./model/bert_dnn_{}.model".format(epoch+1)
#     torch.save(net, model_path)
#     print("saved model: ", model_path)

In [14]:
net = torch.load("./model/bert_dnn_3.model")    # Use the best model we have

In [25]:
s = ["华丽繁荣的城市、充满回忆的小镇、郁郁葱葱的山谷...", "突然就觉得人间不值得"]
tokens = tokenizer(s, padding=True)
input_ids = torch.tensor(tokens["input_ids"])
attention_mask = torch.tensor(tokens["attention_mask"])
last_hidden_states = bert(input_ids, attention_mask=attention_mask)
bert_output = last_hidden_states[0][:, 0]
outputs = net(bert_output)
outputs

tensor([[0.7710],
        [0.1685]], grad_fn=<SigmoidBackward>)

In [26]:
s = ["今天天气真好", "今天天气特别特别棒"]
tokens = tokenizer(s, padding=True)
input_ids = torch.tensor(tokens["input_ids"])
attention_mask = torch.tensor(tokens["attention_mask"])
last_hidden_states = bert(input_ids, attention_mask=attention_mask)
bert_output = last_hidden_states[0][:, 0]
outputs = net(bert_output)
outputs

tensor([[0.9578],
        [0.9773]], grad_fn=<SigmoidBackward>)

In [27]:
# def classify_All(dfRow):
#     tokens = tokenizer(dfRow["text"], padding=True)
#     input_ids = torch.tensor(tokens["input_ids"])
#     attention_mask = torch.tensor(tokens["attention_mask"])
#     last_hidden_states = bert(input_ids, attention_mask=attention_mask)
#     bert_output = last_hidden_states[0][:, 0]
#     outputs = net(bert_output)
#     return outputs

# #dfVerify["bert_result"] = dfVerify.apply(classify, axis = 1)

In [28]:
#classify_All(df)

In [29]:
# def classify_All(dfRow):
#     tokens = tokenizer(dfRow["text"], padding=True)
#     input_ids = torch.tensor(tokens["input_ids"])
#     attention_mask = torch.tensor(tokens["attention_mask"])
#     last_hidden_states = bert(input_ids, attention_mask=attention_mask)
#     bert_output = last_hidden_states[0][:, 0]
#     outputs = net(bert_output)
#     return outputs

# dfPredict["bert_result"] = dfPredict.apply(classify_All, axis = 1)

In [3]:
# url = "https://minio.cvmart.net/user-file/23708/25224f724fdc4f08849ed69877aeab73.csv"
# urllib.request.urlretrieve(url, "verify_data_noWordCut.csv")

('verify_data_noWordCut.csv', <http.client.HTTPMessage at 0x7fe2ef6949e8>)

In [15]:
# dfVerify = pd.read_csv("verify_data_noWordCut.csv")

In [16]:
# display(dfVerify)

,text,label
0,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你],1
1,土耳其的事要认真对待[哈哈]，否则直接开除 很是细心，酒店都全部OK啦。,1
2,姑娘都羡慕你呢…还有招财猫高兴……[哈哈]小学徒一枚，等着明天见您呢大佬范儿[书呆子],1
3,美~~~~~[爱你],1
4,梦想有多大，舞台就有多大![鼓掌],1
...,...,...
119983,一公里不到，县医院那个天桥下右拐200米就到了！ 我靠。这个太霸道了！离224有好远？ 这个...,0
119984,今天真冷啊，难道又要穿棉袄了[晕]？今年的春天真的是百变莫测啊[抓狂],0
119985,最近几天就没停止过！！！[伤心],0
119986,[怒] 很惨!,0


In [ ]:
# tokens = tokenizer(dfVerify["text"].tolist(), padding=True)
# input_ids = torch.tensor(tokens["input_ids"])
# attention_mask = torch.tensor(tokens["attention_mask"])
# last_hidden_states = bert(input_ids, attention_mask=attention_mask)
# bert_output = last_hidden_states[0][:, 0]
# outputs = net(bert_output)

In [ ]:
# def classify_All(dfRow):
#     print(type(dfRow["text"]))
#     tokens = tokenizer(dfRow["text"], padding=True)
#     input_ids = tokens["input_ids"]
#     attention_mask = tokens["attention_mask"]
#     last_hidden_states = bert(input_ids, attention_mask=attention_mask)
#     bert_output = last_hidden_states[0][:, 0]
#     outputs = net(bert_output)
#     return outputs

# dfPredict["bert_result"] = dfPredict.apply(lambda x: classify_All(x), axis = 1)

In [ ]:
# dfVerify["predict"] = outputs.tolist()